Во многих популярных курсах машинного и глубокого обучения вас научат классифицировать собак и кошек, предсказывать цены на недвижимость, покажут еще десятки задач, в которых машинное обучение, вроде как, отлично работает. Но вам расскажут намного меньше (или вообще ничего) о тех случаях, когда ML-модели не работают так, как ожидалось.

Частой проблемой в машинном обучении является неспособность ML-моделей корректно работать на большем разнообразии примеров, чем те, что встречались при обучении. Здесь идет речь не просто о других примерах (например, тестовых), а о других *типах* примеров. Например, сеть обучалась на изображениях коровы, в которых чаще всего корова был на фоне травы, а при тестировании требуется корректное распознавание коровы на любом фоне. Почему ML-модели часто не справляются с такой задачей и что с этим делать - мы рассмотрим далее. Работа над этой проблемой важна не только для решения практических задач, но и в целом для дальнейшего развития ИИ.

~~В большинстве курсов по машинному обучению об этом почему-то не рассказывают. На мой взгляд, начинать ML- и DL-курсы нужно не с простых "hello-world" примеров и не с абстрактной теории, а с конкретных ситуаций, на которых ML-модели *ошибаются*. Это подготовит отличную почву для понимания теории (чтобы хорошо понять теорию, нужно сначала прочувствовать проблемы на практике).~~

Конечно, этим проблемы машинного обучения не ограничиваются, поэтому рассмотрим и другие многочисленные проблемы, такие как сложности с интерпретацией модели, проблемы предвзятости и этики, ресурсоемкости обучения и прочие.

## Проблемы обобщения

### Задача обобщения в машинном обучении

Задача машинного обучения заключается в написании алгоритмов, которые автоматически выводят общие закономерности из частных случаев (обучающих примеров). Этот процесс называется обобщением (*generalization*), или индукцией. Часто требуется найти зависимость между исходными данными ($X$) и целевыми данными ($y$) в виде функции $f: X \to y$.

Например, требуется определить по фотографии тип или положение изображенного объекта. Для решения этой задачи можно собрать большое количество размеченных данных (*supervised learning*), можно также использовать ненадежно размеченные данные (*weakly-supervised learning*), например фотографии из Instagram c хештегами, или даже неразмеченные данные (*self-supervised learning*). Но в любом случае мы используем некий набор конкретных примеров.

Понятно, что чем больше доступно примеров, тем выше будет качество полученного решения  (при прочих равных условиях). Больше данных - выше точность. Все логично?

Существуют даже теоремы, доказывающие для различных ML-алгоритмов стремление получаемого решения к идеалу при неограниченном увеличении количества обучающих данных и размера модели (это свойство называется *universal consistency*; например, для нейронных сетей см. [Faragó and Lugosi, 1993]($Strong Universal Consistency of Neural Network Classifiers$)). При доказательстве подобных теорем считается, что данные $X, y$ берутся из некоего фиксированного, но неизвестного распределения $P(X, y)$. Такой подход называется *statistical learning framework*, он излагается почти в любой книге по машинному обучению (например, см. [Deep Learning book]($Deep Learning$), раздел [5.2](https://www.deeplearningbook.org/contents/ml.html), есть также русское издание).

С развитием технологий стали доступны дешевые вычисления и огромные объемы данных, что позволило эффективно решать очень сложные задачи. Например, языковая модель [GPT-3](https://habr.com/ru/post/514698/) ([Brown et al., 2020]($Language Models are Few-Shot Learners$)) от компании OpenAI обучалась на 570 Гб текстов и имеет 175 миллиардов параметров, по [приблизительным подсчетам](https://www.reddit.com/r/MachineLearning/comments/hwfjej/d_the_cost_of_training_gpt3/) обучение стоило миллионы долларов. Примеры текстов, которые генерирует GPT-3, можно найти [здесь](https://twitter.com/raphaelmilliere/status/1289129723310886912), [здесь](https://twitter.com/raphaelmilliere/status/1289135398644580352), [здесь](https://www.gwern.net/GPT-3) и [здесь](https://tech.liga.net/technology/article/ne-v-moih-interesah-vas-unichtojit-neyroset-gpt-3-naehala-na-tsukerberga-eto-konets). В Google обучили сеть с 1.6 триллиона параметров ([Fedus et al., 2021]($Switch Transformers: Scaling to Trillion Parameter Models with Simple and Efficient Sparsity$)), и это далеко не предел: уже [делаются попытки](https://www.zdnet.com/article/cerebras-prepares-for-the-era-of-120-trillion-parameter-neural-networks/) обучать сеть со 120 триллионами параметров, что примерно равно количеству синапсов в [человеческом мозге](https://insh.world/science/elaborate-visualization-human-brain-date/). Более крупные модели и больше обучающих данных - выше точность.

Казалось бы, что может пойти не так?

Рассмотрим несколько примеров, когда простого увеличения объема обучающих данных может оказаться недостаточно. Начнем с проблемы утечки данных, и затем рассмотрим более общую проблему ограниченного разнообразия обучающих данных и ситуации, когда модель лишь "делает вид", что обучается.

### Утечка данных

Утечкой данных называется ситуация, когда существует некий признак, который при обучении содержал больше информации о целевой переменной, чем при последующем применении модели на практике. Утечка данных может возникать в самых разных формах.

**Пример 1.** При диагностике заболеваний по рентгеновским снимкам модель обучается и тестируется на данных, собранных с разных больниц. Рентгеновские аппараты в разных больницах выдают снимки с немного разной тональностью, при этом различия могут быть даже незаметны человеческим глазом. Модель может обучиться определять больницу на основе тональности снимка, а значит и вероятный диагноз (в разных больницах лежат люди с разными диагнозами), при этом вообще не анализируя изображение на снимке. Такая модель покажет отличную точность при тестировании, так как тестовые снимки взяты из той же выборки данных.

**Пример 2.** В задаче диагностики [реальных и фейковых вакансий](https://www.kaggle.com/shivamb/real-or-fake-fake-jobposting-prediction) почти все фейковые вакансии относились к Европе. Обученная на этих данных модель может не считать подозрительными любые вакансии из других регионов. При применении на практике такая модель станет бесполезна, как только фейковые вакансии станут размещаться в других регионах. Конечно можно удалить регион из признаков, но тогда модель будет пытаться предсказать регион по другим признакам (скажем, по отдельным словам в описании и требованиях), и проблема сохранится.

**Пример 3.** В той же задаче есть признак "имя компании", и у компаний может быть по нескольку вакансий. Если компания фейковая, то все ее вакансии фейковые. Если мы разделим данные на обучающую и тестовую часть (или на фолды) таким образом, что вакансии одной и той же компании попадут и в обучающую, и в тестовую часть, то модели достаточно будет запомнить имена фейковых компаний для предсказания на тестовой выборке, в результате на тесте мы получим сильно завышенную точность.

~~**Пример 4.** Мы хотим реализовать стекинг ([Wolpert, 1992]($Stacked generalization$)), например, обучить модель $B$, которая в качестве входных данных использует как входные данные, так и предсказания модели $A$: $y = B(X, A(X))$. Сначала на выборке $D$ мы обучаем модель $A$, затем на этой же выборке получаем их предсказания, которые используем для обучения модели $B$. Однако обучение как правило сопряжено с переобучением, поэтому ответы модели $A$ на выборке $D$ будут сильнее коррелировать с целевой переменной, чем ответы модели $A$ в среднем по распределению данных. В результате модель $B$ будет полагаться на них в большей степени, чем нужно, что негативно скажется на качестве обучения.~~

Все описанные проблемы нельзя решить просто увеличением количества обучающих данных без изменения подхода к обучению. Здесь можно возразить, что утечка данных не является проблемой самих ML-алгоритмов, поскольку алгоритм и не может знать о том, какой признак нужно учитывать, а какой нет. Но давайте посмотрим на другие примеры, в котором ситуацию, похожую на утечку данных, очень сложно исправить.

### Shortcut learning

В начале XX века жила лошадь по имени Умный Ганс, которая якобы умела решать сложные арифметические задачи, постукивая копытом нужное число раз. В итоге оказалось, что лошадь определяет момент, когда нужно прекращать стучать копытом, по выражению лица того, кто задает вопрос. С тех пор имя "умного Ганса" стало нарицательным ([1]($Unmasking Clever Hans Predictors and Assessing What Machines Really Learn$), [2]($When Choosing Plausible Alternatives, Clever Hans can be Clever$)) - оно означает получение решения обходным, нечестным путем, не решая при этом саму задачу в том смысле, какой мы хотим. Такое обходное "решение" внезапно перестает работать, если меняются условия (например, человек, задающий вопрос лошади, сам не знает ответа на него).

Исследователи давно заметили, что многие современные модели глубокого обучения похожи на "умного Ганса". Вспомним хотя бы легендарную [статью](https://habr.com/ru/post/259191/) на Хабре про "леопардовый диван":

~~> Алгоритм опознал леопарда с достаточной уверенностью, разместив с большим отрывом на втором и третьем месте ягуара и гепарда. ... Как он это делает, черт возьми? ... И вот он тот самый зоологический факт: оказывается, пятна на шкуре этих кошек расположены совсем не случайным образом.~~

> ... Где-то в этот момент в мозг начинает прокрадываться ужасная догадка — а что если вот это различие в текстуре пятен и есть главный критерий, по которому алгоритм отличает три возможных класса распознавания [леопарда, ягуара, гепарда] друг от друга? То есть на самом деле сверточная сеть не обращает внимания на форму изображенного объекта, количество лап, толщину челюсти, особенности позы и все вот эти тонкие различия, которые, как мы было предположили, она умеет понимать — и просто сравнивает картинки как два куска текстуры?

> Это предположение необходимо проверить. Давайте возьмем для проверки простую, бесхитростную картинку, без каких-либо шумов, искажений и прочих факторов, осложняющих жизнь распознаванию. Уверен, эту картинку с первого взгляда легко опознает любой человек. <br> <br> <img src='assets/leopard.jpg' width=250/>

> ... Похоже, что наша догадка насчет текстуры близка к истине. ... Я сделал еще пару проверок (Microsoft, Google) — некоторые из них ведут себя лучше, не подсовывая ягуара, но победить повернутый набок диван не смог никто. Неплохой результат в мире, где уже мелькают заголовки в духе "{Somebody}'s Deep Learning Project Outperforms Humans In Image Recognition".

В самом деле, зачем модели, решающей задачу классификации, выучивать форму и другие комплексные свойства объекта, называемого "леопард", если простое присутствие нужной текстуры на изображении дает почти 100%-ю точность выявления леопарда в обучающей (и тестовой) выборке из ImageNet?

То же самое касается предсказания объекта с учетом фона. Мы не закладывали при обучении никакой информации о том, на основании чего мы хотим получать предсказание: на основании объекта или окружающего его фона. Было даже обнаружено ([Zhu et al., 2016]($Object Recognition with and without Objects$)), что нейронные сети способны с хорошей точностью предсказывать класс объекта, если сам объект вырезан и оставлен только фон.

**Shortcut learning** - явление, когда модели получают верный ответ с помощью неверных (в общем случае) рассуждений, которые хорошо работают только для обучающего распределения данных. Поскольку обучающая и тестовая выборка обычно берутся из одного распределения, то такие модели могут давать хороую точность и при тестировании.

> Shortcuts are decision rules that perform well on standard benchmarks but fail to transfer to more challenging testing conditions, such as real-world scenarios.

Другие примеры можно найти в замечательной статье Shortcut Learning in Deep Neural Networks ([Geirhos et al., 2020]($Shortcut Learning in Deep Neural Networks$)), опубликованной в журнале Nature и основанной на обобщении информации из более чем 140 источников.

Смена угла поворота объекта, появление его в необычном (с точки зрения обучающей выборки) окружении, добавление необычного шума может полностью испортить работу модели. На иллюстрации ниже слева показаны пары изображений, одинаково классифицируемые с точки зрения человека, но совершенно разные с точки зрения сверточной нейронной сети. Справа показаны, наоборот, изображения, имеющие одинаковый класс для нейронной сети и разный для человека.

<img src="assets/generalisation.jpg" width="1000" align="center">

Ссылки на работы, процитированные на иллюстрации:
- [Wang and Deng, 2018]($Deep Visual Domain Adaptation: A Survey$). Deep Visual Domain Adaptation: A Survey.
- [Szegedy et al., 2013](https://arxiv.org/abs/1312.6199). Intriguing properties of neural networks (см. также [обзор]($Intriguing properties of neural networks$)).
- [Dodge and Karam, 2019]($Human and DNN Classification Performance on Images With Quality Distortions: A Comparative Study$). Human and DNN Classification Performance on Images With Quality Distortions: A Comparative Study.
- [Alcorn et al., 2019]($Strike (with) a Pose: Neural Networks Are Easily Fooled by Strange Poses of Familiar Objects$). Strike (with) a Pose: Neural Networks Are Easily Fooled by Strange Poses of Familiar Objects.
- [Geirhos et al., 2019]($ImageNet-trained CNNs are biased towards texture; increasing shape bias improves accuracy and robustness$). ImageNet-trained CNNs are biased towards texture; increasing shape bias improves accuracy and robustness (см. также [обзор](https://habr.com/ru/company/ods/blog/453788/) на Хабре).
- [Beery et al., 2018]($Recognition in Terra Incognita$). Recognition in Terra Incognita.
- [Jacobsen et al., 2019]($Excessive Invariance Causes Adversarial Vulnerability$). Excessive Invariance Causes Adversarial Vulnerability.
- [Nguyen et al., 2015]($Deep Neural Networks are Easily Fooled: High Confidence Predictions for Unrecognizable Images$). Deep Neural Networks are Easily Fooled: High Confidence Predictions for Unrecognizable Images.
- [Hendrycks et al., 2019]($Natural Adversarial Examples$). Natural Adversarial Examples.
- [Brendel and Bethge, 2019]($Approximating CNNs with Bag-of-local-Features models works surprisingly well on ImageNet$). Approximating CNNs with Bag-of-local-Features models works surprisingly well on ImageNet.

### Shortcut learning в языковых моделях

Та же проблема есть и в языковых моделях. Например, модель BERT ([Devlin et al., 2018]($BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding$)) обучена на таком объеме текстов, который человек не прочитает за всю жизнь, но при этом она в основном выучивает поверхностные, стереотипные ассоциации слов друг с другом (word co-occurence), и вряд ли хорошо понимает смысл текста.

> Some of BERT's world knowledge success comes from learning stereotypical associations, e. g. a person with an Italian-sounding name is predicted to be Italian, even when it is incorrect. ([Rogers et al., 2020]($A Primer in BERTology: What we know about how BERT works$))

Попробуем протестировать одни из наиболее популярных на сегодняшний день моделей на основе BERT: RoBERTa ([Liu et al., 2019]($RoBERTa: A Robustly Optimized BERT Pretraining Approach$)) и ALBERT ([Lan et al., 2019]($ALBERT: A Lite BERT for Self-supervised Learning of Language Representations$)), взяв для тестирования их крупные варианты из репозитория HuggingFace: [roberta-large](https://huggingface.co/roberta-large) и [albert-xxlarge-v2](https://huggingface.co/albert-xxlarge-v2). Как и BERT, эти модели обучались на задаче предсказания закрытых маской слов в тексте, называемой masked language model objective, или cloze task (в дополнение, ALBERT использует также sentence-order prediction loss). Проверим их работу на различных примерах. Вы можете сами повторить эксперимент, используя окно "Hosted inference API" по приведенным выше ссылкам.

<img src="assets/mlm.jpg" width="600" align="center">

Как видим, модели неплохо запоминают простые факты, но плохо понимают логику повествования. Если логика повествования противоречит статистике совместной встречаемости слов, то модели обычно предпочитают статистику.

Вероятно, модель ALBERT обучалась на фразах о том, что киты питаются крошечными существами, и поэтому решила, что киты - это крошечные животные. В последних двух примерах модель ALBERT, скорее всего, работает так: если "approaches", то "discuss", если "approach", то "propose" - потому что она часто видела такие сочетания. Слово "existing" при этом ее не смущает. Скорее всего, такая модель вообще нисколько "не понимает" общего значения слов "propose" и "discuss".

Вполне вероятно, что будь модель в 100 раз крупнее (и обучаясь в 100 раз дольше и на большем объеме данных), она не совершила бы таких очевидных ошибок. Но даже эти модели обучались очень долго и на огромных данных, чему качество их работы (на нетипичных примерах) явно не соответствует - это наводит на мысль о неоптимальности подхода в целом.

Проблема shortcut learning присутствует и в тех моделях, которые специально дообучались для [лидербордов](https://gluebenchmark.com/leaderboard) GLUE ([Wang et al., 2018]($GLUE: A Multi-Task Benchmark and Analysis Platform for Natural Language Understanding$)) и SuperGLUE ([Wang et al., 2019]($SuperGLUE: A Stickier Benchmark for General-Purpose Language Understanding Systems$)). Эти бенчмарки нацелены на измерение уровня "понимания естественного языка" моделями машинного обучения. SuperGLUE содержит 8 задач, на некоторых из которых уже побит уровень человека (**TODO имеется в виду средний уровень человека, т. к. разметка тоже делалась людьми**). Но действительно ли уровень человека побит? Как выясняется, высокая метрика качества в SuperGLUE достигается в основном с помощью shortcut learning, при этом модели эксплуатируют систематические проблемы в разметке датасета для предсказания ответа "обходным путем" ([Gururangan et al., 2018]($Annotation Artifacts in Natural Language Inference Data$); [Du et al., 2021]($Towards Interpreting and Mitigating Shortcut Learning Behavior of NLU Models$)).

### Уровни обобщения моделей машинного обобщения

Почти любой датасет имеет ограниченное разнообразие и не покрывает всех ситуаций, в которых желательна корректная работа модели. Особенно это проявляется в случае сложных данных, таких как изображения, тексты и звукозаписи. В данных могут существовать "паразитные корреляции" (*spurious correlations*), позволяющие с хорошей точностью предсказывать ответ только на данной выборке без "комплексного" понимания изображения или текста.

Это как раз и есть рассмотренные выше предсказания на основе фона, отдельных текстурных пятен или отдельных рядом стоящих слов. Датасеты, в которых паразитные корреляции явно выражены, называются **biased** (предвзятыми). Пожалуй, что все датасеты в задачах CV и NLP в той или иной степени предвзяты.

В статье о shortcut learning ([Geirhos et al., 2020]($Shortcut Learning in Deep Neural Networks$)) авторы рассматривают несколько уровней обобщения, которые могут достигать модели машинного обучения:

<img src="assets/taxonomy.jpg" width="340" align="center">

**Uninformative features**. Сеть использует признаки, которые не позволяют эффективно предсказывать ответ даже на обучающей выборке, например, случайно инициализированные веса нейронной сети.

**Overfitting features**. Сеть использует признаки, которые позволяют эффективно предсказывать ответ на обучающей выборке, но не на всем распределении, из которого получена эта выборка. Под "распределением" здесь понимается вероятностное совместное распределение $P(x, y)$, из которого взяты данные (более подробно см. также [этот обзор](https://habr.com/ru/company/ods/blog/599573/), раздел "распределение данных").

**Shortcut features**. Сеть использует признаки, которые позволяют эффективно предсказывать ответ на распределении данных $P(x, y)$, из которого взята обучающая (и, как правило, тестовая) выборка. Поскольку выборка является набором независимых примеров, используется термин *independent and identically distributed ([i. i. d.](https://ru.wikipedia.org/wiki/%D0%9D%D0%B5%D0%B7%D0%B0%D0%B2%D0%B8%D1%81%D0%B8%D0%BC%D1%8B%D0%B5_%D0%BE%D0%B4%D0%B8%D0%BD%D0%B0%D0%BA%D0%BE%D0%B2%D0%BE_%D1%80%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D1%91%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%BB%D1%83%D1%87%D0%B0%D0%B9%D0%BD%D1%8B%D0%B5_%D0%B2%D0%B5%D0%BB%D0%B8%D1%87%D0%B8%D0%BD%D1%8B))*. Способность алгоритма с хорошей точностью работать на некоем фиксированном распределении данных авторы называют **i. i. d. обобщением**. Как мы видели выше, для этого вовсе не обязательно уметь решать саму поставленную задачу в общем виде, часто можно использовать "обходные пути" или утечки данных.

**Intended features**. Сеть использует признаки, которые позволяют эффективно предсказывать ответ в общем случае. Такие признаки будут хорошо работать и вне обучающего распределения, когда "обходные пути" закрыты (например, объект находится на необычном фоне, в необычной позиции, имеет необычную текстуру и пр.). Такая ситуация называется сдвигом распределения данных (*distributional shift*), а соответствующий уровень обобщения называется **out-of-distribution обобщением**. Такой уровень обобщения часто недоступен современным ML-моделям.

Еще один типичный пример - системы распознавания лиц. В обучающем датасете, даже очень большом, может ни быть ни одного изображения человеческого лица, на которое нанесен некий рисунок. Модель, обученная на таком датасете, научится быть инвариантной к положению лица в кадре и его выражению, и почти безошибочно идентифицировать человека. Однако модель может перестать корректно работать в случае, если надеты необычные [очки](https://www.theverge.com/2016/11/3/13507542/facial-recognition-glasses-trick-impersonate-fool) или на лицо нанесен [рисунок](https://www.rbth.com/science_and_tech/2017/07/22/hiding-from-artificial-intelligence-in-the-age-of-total-surveillance_808692), потому что *изображений такого типа не встречалось при обучении*. В результате люди придумывают все новые и новые способы обманывать такие системы, не пряча при этом полностью лицо. Модель не встречалась с такими изображениями при обучении и распознает лица неправильно, даже если человек на месте модели легко опознал бы знакомое лицо.

<img src="assets/facerec.jpg" width="550" align="center">

Конечно, распознавание лиц может не всегда использоваться во благо обществу, но здесь речь идет об общей проблеме систем машинного обучения. Связь этих примеров с *shortcut learning* неочевидна, но явно видна связь с недостатком *out-of-distribution* обобщения, которое можно рассматривать как одну из метрик "интеллектуальности" модели.

Судя по всему, для исправления проблемы *shortcut learning* модель должна уметь представлять входные данные (изображение или текст) как иерархическую комбинацию составляющих частей (объектов, частей тела и т. д.) и быть способной корректно интерпретировать те же части в необычных комбинациях. По сравнению с абстрактным свойством *out-of-distribution обобщения*, это более конкретное желаемое свойство называют *систематическим, или комбинаторным обобщением*. Достижение такого уровня обобщения остается открытой проблемой, о различных подходах к решению которой можно почитать в следующих статьях:

- [Lake and Baroni, 2017]($Generalization without systematicity: On the compositional skills of sequence-to-sequence recurrent networks$). Generalization without systematicity: On the compositional skills of sequence-to-sequence recurrent networks.
-[Battaglia et al., 2018]($Relational inductive biases, deep learning, and graph networks$). Relational inductive biases, deep learning, and graph networks.
- [Bahdanau et al., 2018]($Systematic Generalization: What Is Required and Can It Be Learned?$). Systematic Generalization: What Is Required and Can It Be Learned?
- [Goyal and Bengio, 2020]($Inductive Biases for Deep Learning of Higher-Level Cognition$). Inductive Biases for Deep Learning of Higher-Level Cognition.
- [Hinton, 2021]($How to represent part-whole hierarchies in a neural network$). How to represent part-whole hierarchies in a neural network.

## Другие проблемы машинного обучения

35 реальных рисков, убивающих data- и machine learning проекты https://habr.com/ru/post/649071/ - в основном эти риски сводятся либо к использованию некорректных данных по невнимательности ("мусор на входе - мусор на выходе"), либо к тому, что одни люди создают проблемы другим.

## Заключение